In [31]:
import streamlit as st
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import re
import io

st.title("Croma Product Scraper")

st.markdown("""
Upload a CSV containing product URLs in a column named `URL`. 
The app will scrape Title, Product ID, SKU, MRP, and Price for each product.
""")
input_csv = "products_data.csv"
uploaded_file = st.file_uploader(input_csv, type=["csv"])

if uploaded_file:
    df = pd.read_csv(uploaded_file)
    st.success(f"Loaded {len(df)} URLs from CSV")

    if st.button("Start Scraping"):
        st.info("Scraping in progress... Please wait.")

        # Headless Selenium setup
        options = Options()
        options.add_argument("--headless")
        options.add_argument("--disable-gpu")
        options.add_argument("--window-size=1920,1080")
        options.add_argument("--log-level=3")

        driver = webdriver.Chrome(options=options)

        # Prepare lists
        product_ids = []
        titles = []
        skus = []
        mrps = []
        prices = []

        for url in df['URL']:
            driver.get(url)
            time.sleep(5)

            # ---- Extract Data ----
            try:
                title = driver.find_element(By.CSS_SELECTOR, "h1").text
            except:
                title = "N/A"

            try:
                product_id = re.search(r"/p/(\d+)", driver.current_url).group(1)
            except:
                product_id = "N/A"

            try:
                price = driver.find_element(By.CSS_SELECTOR, "span.amount").text
            except:
                price = "N/A"

            try:
                mrp = driver.find_element(
                    By.XPATH,
                    "/html/body/main/div/div[3]/div/div[1]/div[2]/div[1]/div/div/div/div[3]/div/ul/li[1]/section[1]/div[2]/span[1]"
                ).text
            except:
                mrp = "N/A"

            try:
                sku = driver.find_element(
                    By.XPATH, "//td[contains(text(),'Model Number')]/following-sibling::td"
                ).text
            except:
                match = re.search(r"\((.*?)\)", title)
                sku = match.group(1) if match else "N/A"

            # Convert price and MRP to float
            if price != "N/A":
                price = float(re.sub(r'[^\d.]', '', price.replace(',', '')))
            if mrp != "N/A":
                mrp = float(re.sub(r'[^\d.]', '', mrp.replace(',', '')))

            # Append
            product_ids.append(product_id)
            titles.append(title)
            skus.append(sku)
            mrps.append(mrp)
            prices.append(price)

            st.write(f"Scraped: {title}")

        driver.quit()

        # Update DataFrame
        df['Product ID'] = product_ids
        df['Title'] = titles
        df['SKU'] = skus
        df['MRP'] = mrps
        df['Price'] = prices

        st.success("Scraping completed!")

        # Display updated table
        st.dataframe(df)

        # Download updated CSV
        csv_buffer = io.StringIO()
        df.to_csv(csv_buffer, index=False)
        st.download_button(
            label="Download Updated CSV",
            data=csv_buffer.getvalue(),
            file_name="products_data_updated.csv",
            mime="text/csv"
        )


2025-09-22 18:28:22.772 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-22 18:28:22.773 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-22 18:28:22.773 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-22 18:28:22.774 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-22 18:28:22.775 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-22 18:28:22.775 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-22 18:28:22.776 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-22 18:28:22.776 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar